## Quantile dot plot 

This notebook generates quantile dot plots and 90 days forecast of product categories using output of the model.

In [107]:
import pathlib
import matplotlib.pyplot as plt
import numpy as np
from quantile_dotplot import ntile_dotplot
import matplotlib
import pandas as pd
import json
from matplotlib.ticker import MaxNLocator, FuncFormatter

#### Load Article Data

In [98]:
df_product_cat = pd.read_csv('intermediate_data/Product_Article_Matching.csv')
df_product_cat = df_product_cat.drop(df_product_cat.columns[0], axis=1)
df_product_cat = df_product_cat.reset_index().rename({"index":"id"}, axis=1)
df_product_cat.head(5)

,id,090_day_forecast,Product Category,Products,90 Days Forcast,Product url,Description,Article_1_Score,Article_1_Title,Article_1_Link,Article_2_Score,Article_2_Title,Article_2_Link,Article_3_Score,Article_3_Title,Article_3_Link,dpr_dict
0,0,39.0,Card Guide Accessories,610,23790.0,https://www.digikey.com/en/products/filter/car...,Card Guide Accessories are components used in ...,2.680,10-octave linear-in-pitch VCO with buffered tr...,https://www.edn.com/10-octave-linear-in-pitch-...,NaN,NaN,NaN,NaN,NaN,NaN,"[1.10274622811379, 1.075620195072732, 1.065405..."
1,1,64.0,Wire Wrap,107,6848.0,https://www.digikey.com/en/products/filter/wir...,Wire Wrap is a unique method of connecting ele...,2.471,Multichannel impedance meter analyzes Li-Ion c...,https://www.edn.com/multichannel-impedance-met...,2.210,Robots to enable urban mining of CRMs,https://www.electronicsweekly.com/news/busines...,0.848,Powerline module enables EV charger data links,https://www.edn.com/powerline-module-enables-e...,"[-0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0...."
2,2,31.0,"Electrical, Specialty Fuses",27140,841340.0,https://www.digikey.com/en/products/filter/ele...,Specialty Fuses are specific types of electron...,4.734,Powerline module enables EV charger data links,https://www.edn.com/powerline-module-enables-e...,3.450,Standardization and modularization in pick and...,https://www.edn.com/standardization-and-modula...,2.648,GlobalFoundries Pledges $16 Billion U.S. Inves...,https://www.eetimes.com/globalfoundries-pledge...,"[1.3949346170081829, 1.3972329555288896, 1.437..."
3,3,18.0,Circuit Breakers,73744,1327392.0,https://www.digikey.com/en/products/filter/cir...,Circuit breakers are essential safety devices ...,1.519,A quick and practical view of USB Power Delive...,https://www.edn.com/a-quick-and-practical-view...,1.404,10-octave linear-in-pitch VCO with buffered tr...,https://www.edn.com/10-octave-linear-in-pitch-...,1.207,Intel’s Tech Predictions In 2009,https://www.electronicsweekly.com/blogs/manner...,"[-43.61596367154413, -18.773336606835716, -15...."
4,4,24.0,"Evaluation, Development Board Enclosures",783,18792.0,https://www.digikey.com/en/products/filter/eva...,Evaluation and Development Board Enclosures fo...,4.885,Vehicle hardware security certified to ISO/SAE...,https://www.electronicsweekly.com/news/product...,3.762,Motor gate drivers enable flexible current con...,https://www.edn.com/motor-gate-drivers-enable-...,3.303,Arm Launches Zena to Accelerate AI-Defined Veh...,https://www.eetimes.com/arm-launches-zena-to-a...,"[9587.086862762118, -49.69653554729111, -37.85..."


#### Load Plot Data

In [99]:
df_pred = pd.read_csv("intermediate_data/05_present/Sales_Forcast_Dataframe.csv", index_col=0)
df_pred = df_pred.reset_index().rename({"index":"id"}, axis=1)
df_pred.head(5)

,id,090_day_forecast
0,0,39.0
1,1,64.0
2,2,31.0
3,3,18.0
4,4,24.0


#### Merge Names

In [100]:
df_display = df_pred.merge(df_product_cat, on="id", suffixes=('', '_drop'))
df_display = df_display.loc[:, ~df_display.columns.str.endswith('_drop')] # Drop duplicate columns
df_display.head()

,id,090_day_forecast,Product Category,Products,90 Days Forcast,Product url,Description,Article_1_Score,Article_1_Title,Article_1_Link,Article_2_Score,Article_2_Title,Article_2_Link,Article_3_Score,Article_3_Title,Article_3_Link,dpr_dict
0,0,39.0,Card Guide Accessories,610,23790.0,https://www.digikey.com/en/products/filter/car...,Card Guide Accessories are components used in ...,2.680,10-octave linear-in-pitch VCO with buffered tr...,https://www.edn.com/10-octave-linear-in-pitch-...,NaN,NaN,NaN,NaN,NaN,NaN,"[1.10274622811379, 1.075620195072732, 1.065405..."
1,1,64.0,Wire Wrap,107,6848.0,https://www.digikey.com/en/products/filter/wir...,Wire Wrap is a unique method of connecting ele...,2.471,Multichannel impedance meter analyzes Li-Ion c...,https://www.edn.com/multichannel-impedance-met...,2.210,Robots to enable urban mining of CRMs,https://www.electronicsweekly.com/news/busines...,0.848,Powerline module enables EV charger data links,https://www.edn.com/powerline-module-enables-e...,"[-0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0...."
2,2,31.0,"Electrical, Specialty Fuses",27140,841340.0,https://www.digikey.com/en/products/filter/ele...,Specialty Fuses are specific types of electron...,4.734,Powerline module enables EV charger data links,https://www.edn.com/powerline-module-enables-e...,3.450,Standardization and modularization in pick and...,https://www.edn.com/standardization-and-modula...,2.648,GlobalFoundries Pledges $16 Billion U.S. Inves...,https://www.eetimes.com/globalfoundries-pledge...,"[1.3949346170081829, 1.3972329555288896, 1.437..."
3,3,18.0,Circuit Breakers,73744,1327392.0,https://www.digikey.com/en/products/filter/cir...,Circuit breakers are essential safety devices ...,1.519,A quick and practical view of USB Power Delive...,https://www.edn.com/a-quick-and-practical-view...,1.404,10-octave linear-in-pitch VCO with buffered tr...,https://www.edn.com/10-octave-linear-in-pitch-...,1.207,Intel’s Tech Predictions In 2009,https://www.electronicsweekly.com/blogs/manner...,"[-43.61596367154413, -18.773336606835716, -15...."
4,4,24.0,"Evaluation, Development Board Enclosures",783,18792.0,https://www.digikey.com/en/products/filter/eva...,Evaluation and Development Board Enclosures fo...,4.885,Vehicle hardware security certified to ISO/SAE...,https://www.electronicsweekly.com/news/product...,3.762,Motor gate drivers enable flexible current con...,https://www.edn.com/motor-gate-drivers-enable-...,3.303,Arm Launches Zena to Accelerate AI-Defined Veh...,https://www.eetimes.com/arm-launches-zena-to-a...,"[9587.086862762118, -49.69653554729111, -37.85..."


#### Load Dot Plot Ratio

In [101]:
with open('intermediate_data/05_present/Dot_Plot_Ratio.json', 'r') as json_file:
    cat_dotplot_ratio_dict = json.load(json_file)
cat_dotplot_ratio_dict["0"][:10]

[1.10274622811379,
 1.075620195072732,
 1.065405619094332,
 1.0418655445437526,
 1.0472306270086833,
 1.073338505707987,
 1.0943292592541056,
 1.0155921947057949,
 1.0308640289342124,
 1.064380247570561]

#### Merge Datasets
Add in Dotplot Ratio Dict

In [102]:
df_display["dpr_dict"] = df_display["id"].apply(lambda x: cat_dotplot_ratio_dict[str(x)])
df_display.head()

,id,090_day_forecast,Product Category,Products,90 Days Forcast,Product url,Description,Article_1_Score,Article_1_Title,Article_1_Link,Article_2_Score,Article_2_Title,Article_2_Link,Article_3_Score,Article_3_Title,Article_3_Link,dpr_dict
0,0,39.0,Card Guide Accessories,610,23790.0,https://www.digikey.com/en/products/filter/car...,Card Guide Accessories are components used in ...,2.680,10-octave linear-in-pitch VCO with buffered tr...,https://www.edn.com/10-octave-linear-in-pitch-...,NaN,NaN,NaN,NaN,NaN,NaN,"[1.10274622811379, 1.075620195072732, 1.065405..."
1,1,64.0,Wire Wrap,107,6848.0,https://www.digikey.com/en/products/filter/wir...,Wire Wrap is a unique method of connecting ele...,2.471,Multichannel impedance meter analyzes Li-Ion c...,https://www.edn.com/multichannel-impedance-met...,2.210,Robots to enable urban mining of CRMs,https://www.electronicsweekly.com/news/busines...,0.848,Powerline module enables EV charger data links,https://www.edn.com/powerline-module-enables-e...,"[-0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0...."
2,2,31.0,"Electrical, Specialty Fuses",27140,841340.0,https://www.digikey.com/en/products/filter/ele...,Specialty Fuses are specific types of electron...,4.734,Powerline module enables EV charger data links,https://www.edn.com/powerline-module-enables-e...,3.450,Standardization and modularization in pick and...,https://www.edn.com/standardization-and-modula...,2.648,GlobalFoundries Pledges $16 Billion U.S. Inves...,https://www.eetimes.com/globalfoundries-pledge...,"[1.3949346170081829, 1.3972329555288896, 1.437..."
3,3,18.0,Circuit Breakers,73744,1327392.0,https://www.digikey.com/en/products/filter/cir...,Circuit breakers are essential safety devices ...,1.519,A quick and practical view of USB Power Delive...,https://www.edn.com/a-quick-and-practical-view...,1.404,10-octave linear-in-pitch VCO with buffered tr...,https://www.edn.com/10-octave-linear-in-pitch-...,1.207,Intel’s Tech Predictions In 2009,https://www.electronicsweekly.com/blogs/manner...,"[-43.61596367154413, -18.773336606835716, -15...."
4,4,24.0,"Evaluation, Development Board Enclosures",783,18792.0,https://www.digikey.com/en/products/filter/eva...,Evaluation and Development Board Enclosures fo...,4.885,Vehicle hardware security certified to ISO/SAE...,https://www.electronicsweekly.com/news/product...,3.762,Motor gate drivers enable flexible current con...,https://www.edn.com/motor-gate-drivers-enable-...,3.303,Arm Launches Zena to Accelerate AI-Defined Veh...,https://www.eetimes.com/arm-launches-zena-to-a...,"[9587.086862762118, -49.69653554729111, -37.85..."


#### Predicts 90 days forecast

In [103]:
# Calculate adjusted prediction for each product
def calculate_adjusted_prediction(row):
    prediction = row["090_day_forecast"]
    products_count = row["Products"] 
    return prediction * products_count

# Modify output
df_display["90 Days Forecast"] = df_display.apply(calculate_adjusted_prediction, axis=1)  # Add adjusted_prediction as the 4th column
cols = df_display.columns.tolist() # Reorder columns to make adjusted_prediction the 4th column
cols.remove('90 Days Forecast') # Remove adjusted_prediction from its current position
cols.insert(4, '90 Days Forecast') # Insert it at position 3 (4th column, 0-indexed)
df_display = df_display[cols]

# Save the updated dataframe back to CSV
df_display.to_csv('intermediate_data/Product_Article_Matching.csv', index=False)

#### Create dot plot graph using prediction

In [104]:
cat_specific = "Plug Housings"

def dotplot_sku_function(df , cat):
    plt.figure(figsize=(8, 6))  #
    filtered_df = df[df["Product Category"] == cat]
    if filtered_df.empty:
        print(f"No data found for category: '{cat}' - skipping")
        return

    data = df[df["Product Category"] == cat]["dpr_dict"].iloc[0]
    adjusted_prediction = df[df["Product Category"] == cat]["90 Days Forecast"].iloc[0]
    dotplotdata = np.abs(adjusted_prediction * np.array(data))

    #dotplotdata = np.abs(prediction * np.array(data))
    date_latest = "Jun/11/2025"
    dotplt = ntile_dotplot(dotplotdata, dots=15, hist_bins="auto")
    print(type(dotplt))
    dotplt.axvline(x=adjusted_prediction, color='r', linestyle='--', linewidth=2, label='Prediction')
    dotplt.set_title("90 Day Sales Prediction for \n {} \n on {} ".format(cat, date_latest),
                                                                       fontsize=20)
    #, fontweight='bold')
    dotplt.set_xlabel("\n Forecast Unit Sales Next 90 Days", fontsize=20)

    # Remove top, right, and left spines, keep only bottom axis
    dotplt.spines['top'].set_visible(False)
    dotplt.spines['right'].set_visible(False)
    dotplt.spines['left'].set_visible(False)
    
    # Remove y-axis ticks and labels
    dotplt.set_yticks([])
    dotplt.set_ylabel('')

    # Format values on x-axis for consistency
    dotplt.xaxis.set_major_locator(MaxNLocator(nbins=6, integer=True))
    dotplt.xaxis.set_major_formatter(FuncFormatter(lambda x, p: f'{x:.0f}'))
    dotplt.tick_params(axis='x', labelsize=15) 
    dotplt.figure.savefig("figures/sfp_{}.png".format(cat.translate(str.maketrans(" /", "__"))), 
                          bbox_inches='tight', dpi=300, pad_inches=0.2)

#### Run method for one dataset

In [105]:
dotplot_sku_function(df_display, cat_specific)

No data found for category: 'Plug Housings' - skipping


<Figure size 800x600 with 0 Axes>

#### Run method for all datasets

In [106]:
for cat_one in df_display["Product Category"].unique():
    #print(cat_one)
    dotplot_sku_function(df_display, cat_one)
    plt.clf()

<class 'matplotlib.axes._axes.Axes'>
<class 'matplotlib.axes._axes.Axes'>
<class 'matplotlib.axes._axes.Axes'>
<class 'matplotlib.axes._axes.Axes'>
<class 'matplotlib.axes._axes.Axes'>
<class 'matplotlib.axes._axes.Axes'>
<class 'matplotlib.axes._axes.Axes'>
<class 'matplotlib.axes._axes.Axes'>
<class 'matplotlib.axes._axes.Axes'>
<class 'matplotlib.axes._axes.Axes'>
<class 'matplotlib.axes._axes.Axes'>
<class 'matplotlib.axes._axes.Axes'>
<class 'matplotlib.axes._axes.Axes'>
<class 'matplotlib.axes._axes.Axes'>
<class 'matplotlib.axes._axes.Axes'>
<class 'matplotlib.axes._axes.Axes'>
<class 'matplotlib.axes._axes.Axes'>
<class 'matplotlib.axes._axes.Axes'>
<class 'matplotlib.axes._axes.Axes'>
<class 'matplotlib.axes._axes.Axes'>


/var/folders/yk/12gxrg_j4_7fdrzzrf8kxwr00000gn/T/ipykernel_22080/537445867.py:4: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  plt.figure(figsize=(8, 6))  #


<class 'matplotlib.axes._axes.Axes'>
<class 'matplotlib.axes._axes.Axes'>
<class 'matplotlib.axes._axes.Axes'>
<class 'matplotlib.axes._axes.Axes'>
<class 'matplotlib.axes._axes.Axes'>
<class 'matplotlib.axes._axes.Axes'>
<class 'matplotlib.axes._axes.Axes'>
<class 'matplotlib.axes._axes.Axes'>
<class 'matplotlib.axes._axes.Axes'>
<class 'matplotlib.axes._axes.Axes'>
<class 'matplotlib.axes._axes.Axes'>
<class 'matplotlib.axes._axes.Axes'>
<class 'matplotlib.axes._axes.Axes'>
<class 'matplotlib.axes._axes.Axes'>
<class 'matplotlib.axes._axes.Axes'>
<class 'matplotlib.axes._axes.Axes'>
<class 'matplotlib.axes._axes.Axes'>
<class 'matplotlib.axes._axes.Axes'>
<class 'matplotlib.axes._axes.Axes'>
<class 'matplotlib.axes._axes.Axes'>
<class 'matplotlib.axes._axes.Axes'>
<class 'matplotlib.axes._axes.Axes'>
<class 'matplotlib.axes._axes.Axes'>
<class 'matplotlib.axes._axes.Axes'>
<class 'matplotlib.axes._axes.Axes'>
<class 'matplotlib.axes._axes.Axes'>
<class 'matplotlib.axes._axes.Axes'>
<

<Figure size 576x432 with 0 Axes>

<Figure size 576x432 with 0 Axes>

...

<Figure size 576x432 with 0 Axes>

<Figure size 576x432 with 0 Axes>

<Figure size 576x432 with 0 Axes>

<Figure size 576x432 with 0 Axes>

<Figure size 576x432 with 0 Axes>